In [ ]:
import polars as pl
from time import perf_counter

t_start = perf_counter()
# Read file as string
data: str
with open('samples.json', 'r') as file:
    data = file.read()

t2 = perf_counter()

result = pl.LazyFrame({"arrs": data}).select(pl.col("arrs").str.json_extract(pl.List(pl.List(pl.UInt8))).explode()).with_columns(
    id= pl.int_range(0, pl.col('arrs').count())
).explode("arrs").group_by('id').agg(pl.col('arrs').value_counts(parallel=True)).explode("arrs").unnest('arrs').group_by('id').agg([
    pl.col('arrs').filter(pl.col('counts').mod(2) == 1)
]).explode('arrs').collect().get_column('arrs').to_list()

# Stop the timer
t3 = perf_counter()

print(result)

t_stop = perf_counter()

print(f"File reading {round((t2-t_start) * 1000, 2)}ms")
print(f"Compute {round((t3-t2) * 1000, 2)}ms")
print(f"Print {round((t_stop-t3) * 1000, 2)}ms")


In [ ]:
data: str
with open('samples.json', 'r') as file:
    data = file.read()

In [ ]:
parsed_json = pl.DataFrame({"arrs": data}).select(pl.col("arrs").str.json_extract(pl.List(pl.List(pl.UInt8))).explode())
print(parsed_json)

In [ ]:
ids = parsed_json.with_columns(
    id= pl.int_range(0, pl.col('arrs').count())
)
print(ids)

In [ ]:
counts = ids.explode("arrs").group_by('id').agg(pl.col('arrs').value_counts(parallel=True)).explode("arrs").unnest('arrs')
print(counts)

In [ ]:
find_odds = counts.group_by('id').agg([pl.col('arrs').filter(pl.col('counts').mod(2) == 1)])
print(find_odds)

In [ ]:
result = find_odds.explode('arrs').get_column('arrs').to_list()
print(result)